# Pandas / Ibis / deltalake

In [2]:
import os

import pandas as pd
from deltalake import DeltaTable
from deltalake.writer import write_deltalake

# Create a Delta table

In [3]:
df = pd.DataFrame({"x": ["a", "b", "c", "a"], "y": [1, 2, 3, 4]})

In [4]:
df

,x,y
0,a,1
1,b,2
2,c,3
3,a,4


In [5]:
os.makedirs("tmp/some_delta_lake", exist_ok=True)

In [6]:
write_deltalake("tmp/some_delta_lake", df)

## Append to the Delta table

In [7]:
df = pd.DataFrame({"x": ["l", "m"], "y": [20, 30]})

In [8]:
df

,x,y
0,l,20
1,m,30


In [9]:
write_deltalake("tmp/some_delta_lake", df, mode="append")

## Create an Ibis table from the Delta table

In [10]:
import ibis

In [12]:
con = ibis.pandas.connect({"t": DeltaTable("tmp/some_delta_lake").to_pandas()})

In [13]:
con.list_tables()

['t']

In [14]:
t = con.table("t")

In [18]:
t.head(2).execute()

,x,y
0,a,1
1,b,2


In [21]:
t.filter(t.y >= 4).execute()

,x,y
0,a,4
1,l,20
2,m,30


## Time travel back to version 0 of the data

In [22]:
con = ibis.pandas.connect({"t": DeltaTable("tmp/some_delta_lake", version=0).to_pandas()})

In [23]:
t = con.table("t")

In [24]:
t.filter(t.y >= 4).execute()

,x,y
0,a,4
